In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, LSTM, ConvLSTM2D, BatchNormalization
from keras.layers import Concatenate, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import regularizers
from keras import backend as K
import keras.losses

import tensorflow as tf

import numpy as np

import sys
import os


Using TensorFlow backend.


In [2]:
#Load isoform legacy model (theano weight files)
legacy_model_name = 'generalapa_sparse_general_global_onesided2antimisprimeorigdropout_finetuned_TOMM5_APA_Six_30_31_34_default'

legacy_iso_conv_layer_1_w = np.load(legacy_model_name + '_conv0_left_w.npy')
legacy_iso_conv_layer_1_b = np.load(legacy_model_name + '_conv0_left_b.npy')

legacy_iso_conv_layer_2_w = np.load(legacy_model_name + '_conv1_w.npy')
legacy_iso_conv_layer_2_b = np.load(legacy_model_name + '_conv1_b.npy')

legacy_iso_dense_layer_1_w = np.load(legacy_model_name + '_mlp_w.npy')
legacy_iso_dense_layer_1_b = np.load(legacy_model_name + '_mlp_b.npy')

legacy_iso_out_layer_w = np.load(legacy_model_name + '_lr_w.npy')
legacy_iso_out_layer_b = np.load(legacy_model_name + '_lr_b.npy')

#Load cut legacy model (theano weight files)
legacy_cut_model_name = 'generalapa_sparse_general_global_onesided2cuts2antimisprimeorigdropout_finetuned_TOMM5_APA_Six_30_31_34_default'#_pasaligned

legacy_cut_conv_layer_1_w = np.load(legacy_cut_model_name + '_conv0_left_w.npy')
legacy_cut_conv_layer_1_b = np.load(legacy_cut_model_name + '_conv0_left_b.npy')

legacy_cut_conv_layer_2_w = np.load(legacy_cut_model_name + '_conv1_w.npy')
legacy_cut_conv_layer_2_b = np.load(legacy_cut_model_name + '_conv1_b.npy')

legacy_cut_dense_layer_1_w = np.load(legacy_cut_model_name + '_mlp_w.npy')
legacy_cut_dense_layer_1_b = np.load(legacy_cut_model_name + '_mlp_b.npy')

legacy_cut_out_layer_w = np.load(legacy_cut_model_name + '_lr_w.npy')
legacy_cut_out_layer_b = np.load(legacy_cut_model_name + '_lr_b.npy')


In [3]:
#APARENT parameters
seq_input_shape = (1, 185, 4)
lib_input_shape = (36,)
distal_pas_shape = (1,)


#Isoform model definition
iso_layer_1 = Conv2D(70, (8, 4), padding='valid', activation='relu', kernel_initializer='zeros', data_format='channels_first', name='iso_conv_layer_1')
iso_layer_1_pool = MaxPooling2D(pool_size=(2, 1), data_format='channels_first', name='iso_maxpool_layer_1')
iso_layer_2 = Conv2D(110, (6, 1), padding='valid', activation='relu', kernel_initializer='zeros', data_format='channels_first', name='iso_conv_layer_2')
iso_layer_dense = Dense(80, activation='relu', kernel_initializer='zeros', name='iso_dense_layer_1')
iso_layer_drop = Dropout(0.2, name='iso_drop_layer_1')
iso_layer_out = Dense(2, activation='softmax', kernel_initializer='zeros', name='iso_out_layer_1')

def iso_model(seq_input, distal_pas_input, lib_input) :
    return iso_layer_out(
        Concatenate()([
            iso_layer_drop(
                iso_layer_dense(
                    Concatenate()([
                        Flatten()(
                            iso_layer_2(
                                iso_layer_1_pool(
                                    iso_layer_1(
                                        seq_input
                                    )
                                )
                            )
                        ),
                        distal_pas_input
                    ])
                )
            ),
            lib_input
        ])
    )

#Cut model definition
cut_layer_1 = Conv2D(70, (8, 4), padding='valid', activation='relu', kernel_initializer='zeros', data_format='channels_first', name='cut_conv_layer_1')
cut_layer_1_pool = MaxPooling2D(pool_size=(2, 1), data_format='channels_first', name='cut_maxpool_layer_1')
cut_layer_2 = Conv2D(110, (6, 1), padding='valid', activation='relu', kernel_initializer='zeros', data_format='channels_first', name='cut_conv_layer_2')
cut_layer_dense = Dense(400, activation='relu', kernel_initializer='zeros', name='cut_dense_layer_1')
cut_layer_drop = Dropout(0.2, name='cut_drop_layer_1')
cut_layer_out = Dense(186, activation='softmax', kernel_initializer='zeros', name='cut_out_layer_1')

def cut_model(seq_input, distal_pas_input, lib_input) :
    return cut_layer_out(
        Concatenate()([
            cut_layer_drop(
                cut_layer_dense(
                    Concatenate()([
                        Flatten()(
                            cut_layer_2(
                                cut_layer_1_pool(
                                    cut_layer_1(
                                        seq_input
                                    )
                                )
                            )
                        ),
                        distal_pas_input
                    ])
                )
            ),
            lib_input
        ])
    )


#Join model definition

#Inputs
seq_input = Input(shape=seq_input_shape)
lib_input = Input(shape=lib_input_shape)
distal_pas_input = Input(shape=distal_pas_shape)
batch_size = K.shape(seq_input)[0]

out_iso = iso_model(seq_input, distal_pas_input, lib_input)
out_cut = cut_model(seq_input, distal_pas_input, lib_input)

out_iso_trimmed = Lambda(lambda x: K.expand_dims(x[:, 1], axis=-1), output_shape=(1,))(out_iso)

plasmid_model = Model(
    inputs=[
        seq_input,
        lib_input,
        distal_pas_input
    ],
    outputs=[
        out_iso_trimmed,
        out_cut
    ]
)

In [4]:
#Transfer legacy theano weights to the new keras model

#Transfer isoform model weights
legacy_iso_conv_layer_1_w_flipped = legacy_iso_conv_layer_1_w[:, :, ::-1, ::-1]
legacy_iso_conv_layer_2_w_flipped = legacy_iso_conv_layer_2_w[:, :, ::-1, ::-1]
legacy_iso_conv_layer_1_w_permuted = np.moveaxis(np.moveaxis(legacy_iso_conv_layer_1_w_flipped, 0, -1), 0, 2)
legacy_iso_conv_layer_2_w_permuted = np.moveaxis(np.moveaxis(legacy_iso_conv_layer_2_w_flipped, 0, -1), 0, 2)

iso_layer_1.set_weights([legacy_iso_conv_layer_1_w_permuted, legacy_iso_conv_layer_1_b])
iso_layer_2.set_weights([legacy_iso_conv_layer_2_w_permuted, legacy_iso_conv_layer_2_b])
iso_layer_dense.set_weights([legacy_iso_dense_layer_1_w, legacy_iso_dense_layer_1_b])
iso_layer_out.set_weights([legacy_iso_out_layer_w, legacy_iso_out_layer_b])

#Transfer cut model weights
legacy_cut_conv_layer_1_w_flipped = legacy_cut_conv_layer_1_w[:, :, ::-1, ::-1]
legacy_cut_conv_layer_2_w_flipped = legacy_cut_conv_layer_2_w[:, :, ::-1, ::-1]
legacy_cut_conv_layer_1_w_permuted = np.moveaxis(np.moveaxis(legacy_cut_conv_layer_1_w_flipped, 0, -1), 0, 2)
legacy_cut_conv_layer_2_w_permuted = np.moveaxis(np.moveaxis(legacy_cut_conv_layer_2_w_flipped, 0, -1), 0, 2)

cut_layer_1.set_weights([legacy_cut_conv_layer_1_w_permuted, legacy_cut_conv_layer_1_b])
cut_layer_2.set_weights([legacy_cut_conv_layer_2_w_permuted, legacy_cut_conv_layer_2_b])
cut_layer_dense.set_weights([legacy_cut_dense_layer_1_w, legacy_cut_dense_layer_1_b])
cut_layer_out.set_weights([legacy_cut_out_layer_w, legacy_cut_out_layer_b])


In [5]:

save_dir = '.'
new_legacy_model_name = 'aparent_theano_legacy_30_31_34.h5'#'aparent_theano_legacy_30_31_34_pasaligned.h5'

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, new_legacy_model_name)
plasmid_model.save(model_path)
print('Saved trained model at %s ' % model_path)


Saved trained model at ./aparent_theano_legacy_30_31_34.h5 
